In [814]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
from typing import Tuple, List

rd.seed(6666) # devil with extra digit

# Project Idea: Find the (Non-Complex) Eigenvalue of a Matrix

It's from scratch-ish with some use of `numpy` because I don't want to write a hundred for-loops.

## References

* Lots of videos by **Mu Prime Math** ([used playlist](https://www.youtube.com/playlist?list=PLug5ZIRrShJHNCfEiX6l5CKbljWayGEcs))
* [Also this video](https://www.youtube.com/watch?v=LHlg_lfihiA)
* [Also this link](https://math.nyu.edu/~stadler/num1/material/num1_eigenvalues.pdf)

## Power Method

An iterative algorithm that finds the eigenvalue with the highest magnitude.

### Update Rule

$$ b_{k+1} = \frac{Ab_k}{||Ab_k||} $$

such that

* $A$ is an $n\times n$ matrix
* $b_k$ is an $n$-element vector
* $||Ab_k||$ is the element with the highest magnitude of $Ab_k$
* $b_0$ is non-zero
* Stop when $b_k = b_{k+1}$ (once the scaled $b_{k+1}$ is the same as $b_k$)

Once convergence is reached, we will end up with an eigenvector of $A$ that corresponds to the 
eigenvalue with the biggest magnitude. To find the eigenvalue from this, we can scale down the
eigenvector $\vec{v}$ so the largest element is 1, multiply $\vec{v}$ by $A$,
then the eigenvalue is the element that has the highest magnitude.

Note: The code doesn't do this exactly but the code looks cleaner this way.

### Proof

Let's say we have matrix $A$ with eigenvectors $\vec{v_1}, \vec{v_2}, \ldots, \vec{v_n}$ with 
corresponding eigenvalues $\lambda_1, \lambda_2, \ldots, \lambda_n$.

Let $\vec{x}$ be written as a linear combination of the eigenvectors of $A$.

$$ \vec{x} = c_1\vec{v_1} + c_2\vec{v_2} + \ldots + c_n\vec{v_n} $$

We can then multiply $A$ to both sides, giving

$$ A\vec{x} = Ac_1\vec{v_1} + Ac_2\vec{v_2} + \ldots + Ac_n\vec{v_n} $$

If we keep multiplying $A$ to both sides (let's say $k$ times), we will end up with

$$ A^k\vec{x} = A^kc_1\vec{v_1} + A^kc_2\vec{v_2} + \ldots + A^kc_n\vec{v_n} $$

Because $\vec{v_1}, \vec{v_2}, \ldots, \vec{v_n}$ are eigenvectors of $A$, we know that $A\vec{v_i} = \lambda_i\vec{v_i} $.
We also know that if $\lambda$ is an eigenvalue of $A$, $\lambda^k$ would be the eigenvalue of $A^k$.
Thus, we get

$$ A^k\vec{x} = c_1\lambda_1^k\vec{v_1} + c_2\lambda_2^k\vec{v_2} + \ldots + c_n\lambda_n^k\vec{v_n} $$

Let $\lambda_1$ be the eigenvalue of $A$ with the highest magnitude. We can factor it out to get

$$ A^k\vec{x} = \lambda_1^k\left(c_1\vec{v_1} + \frac{c_2\lambda_2^k\vec{v_2}}{\lambda_1^k} + \ldots + \frac{c_n\lambda_n^k\vec{v_n}}{\lambda_1^k}\right) $$

Because $|\lambda_1| > |\lambda_2|, \ldots, |\lambda_n|$,

$$\lim_{k\to\infty} \frac{\lambda_i^k}{\lambda_1^k} = 0~\forall i\neq1 $$

Therefore, we will end up with this when $k$ is very big

$$ A^k\vec{x} =  c_1 \lambda_1^k \vec{v_1} $$

This means that we can just choose an arbitrary $\vec{x}$ to find the eigenvalue with the
highest magnitude and its corresponding eigenvector as long as we multiply $A$ enough times.

We will end up with a scaled version of an eigenvector, denoted as $c\vec{v}$ for some constant $c$.

For the finding eigenvalue part, this is because $A\vec{v} = \lambda{v}$. Therefore, $\lambda\times1 = \lambda$

### Code

In [815]:
def power_eig(A:np.ndarray) -> Tuple[float, np.ndarray]:
	size, _ = A.shape
	A = A.astype(float)
	vec = np.ones(size).astype(float)
	pvec = np.zeros(size).astype(float)

	while (vec != pvec).all():
		pvec = vec
		imax = np.argmax(np.abs(vec))
		vec = np.matmul(A, vec) / vec[imax]

	return vec[imax], vec/vec[imax]

In [816]:
test_matrix = np.array([[5., 8., 16.],
						[4., 1., 8.],
						[-4., -4., -11.]])
max_eival, max_eivec = power_eig(test_matrix)
print(max_eival, max_eivec)

-3.0000000000000018 [ 1.          0.42857143 -0.71428571]


In [817]:
print("eigenvector:\t", max_eivec)
print("A*eivec:\t", np.matmul(test_matrix, max_eivec))
print("eival*eivec:\t", max_eival * max_eivec)

eigenvector:	 [ 1.          0.42857143 -0.71428571]
A*eivec:	 [-3.         -1.28571429  2.14285714]
eival*eivec:	 [-3.         -1.28571429  2.14285714]


## Shifted Inverse Power Method

An iterative algorithm for finding eigenvalues that isn't the biggest one.

The problem with the power method is that it only
works for finding the eigenvalue and eigenvector for the eigenvalue that has the highest magnitude
due to it *dominating* the other terms as shown above.

Thus, we have to modify the power method so that it works with finding the other eigenvalues of the
using the shifted inverse power method.

### Update Rule

$$ b_{k+1} = \frac{(A - \alpha I)^{-1}b_k}{||(A - \alpha I)^{-1}b_k||} $$

such that:
* $\alpha$ is the initial guess for the eigenvalue; "*the shift*"
* The rest are similar to the regular power method
* Gives us the eigenvalue that is closest to $\alpha$ and its corresponding eigenvector

Once convergence is reached, we'll end up with eigenvector $\vec{v}$ that is an eigenvector of $A$
which happens to correspond to $(\lambda - \alpha)^{-1}$.

Let $\omega = (\lambda - \alpha)^{-1} = \frac{1}{\lambda - \alpha}$,

$$ \lambda = \frac{1}{\omega} + \alpha $$

where $\lambda$ is the eigenvalue that is closest to $\alpha$.

### Proof

Recall the definition of eigenvalue and eigenvector of matrix $A$:

$$ A\vec{v} = \lambda\vec{v} $$

The following parts will seem like it came out of thin air. That's because it is. Bare with me.

We can subtract $\alpha\vec{v}$ from both sides
in the appropriate formats

$$ A\vec{v} - \alpha I\vec{v} = \lambda\vec{v} - \alpha\vec{v} $$

We can then simplify it down to 

$$ (A - \alpha I)\vec{v} = (\lambda - \alpha)\vec{v} $$

Then, we can multiply the inverse of $A - \alpha I$ to both sides then divide both sides by
$\lambda - \alpha$ to get

$$ (A - \alpha I)^{-1}\vec{v} = \frac{1}{\lambda - \alpha}\vec{v} \equiv (\lambda - \alpha)^{-1}\vec{v} $$

This is basically the definition of eigenvalue-vector as well but for $(A - \alpha I)^{-1}$.
However, $\vec{v}$ will still be the eigenvector of $A$ as well because this whole thing was based
on the "typical" definition of the matrix. We just did a bit of manipulation on them.

Now, we can use the power method on this *fancy new matrix* and its *fancy new value*.

Let $$ B = (A - \alpha I)^{-1} , \omega = (\lambda - \alpha)^{-1} $$
where $\omega$ is the eigenvalue of $B$ with the highest magnitude.

Let us write $\vec{x}$ as a linear combination of eigenvectors $\vec{v}_1, \ldots, \vec{v}_n$ of $B$.

$$ \vec{x} = c_1\vec{v_1} + \ldots + c_n\vec{v_n} $$

Since this is similar to the regular power method, let us speed through this. We can multiply $B$
to both sides $k$ times and we'll eventually end up with this when $k$ is very big

$$ B^k\vec{x} = c\omega^k\vec{v} $$

Finally, we can do algebra to convert it back to fit $A$.

As for why $\lambda$ is the eigenvalue closest to $\alpha$, here's the explanation.

For the power method, the reason that we are able to find the eigenvalue $\lambda$ with the largest magnitude
from it is because the value of $\lambda$ would dominate the rest with enough multiplications.

When we change from $\lambda$ by itself to $\frac{1}{\lambda - \alpha}$, this means that the closer
$\lambda$ is to $\alpha$, the bigger the term will be. Thus, it will become the dominating term instead.

### Code

In [818]:
def shifted_eig(A:np.ndarray, alpha:float):
	size, _ = A.shape
	vec = np.ones(size).astype(float)
	pvec = np.zeros(size).astype(float)
	A = A.astype(float)
	inv = np.linalg.inv(A - alpha*np.identity(size))

	while (vec != pvec).all():
		pvec = vec
		imax = np.argmax(np.abs(vec))
		vec = np.matmul(inv, vec) / vec[imax]

	return np.round(1/vec[imax]+alpha, 6), vec/vec[imax]

In [819]:
eival, eivec = shifted_eig(test_matrix, 1.1)
print(eival, eivec)

1.0 [ 1.   0.5 -0.5]


In [820]:
print("eigenvector:\t", eivec)
print("A*eivec:\t", np.matmul(test_matrix, eivec))
print("eival*eivec:\t", eival * eivec)

eigenvector:	 [ 1.   0.5 -0.5]
A*eivec:	 [ 1.   0.5 -0.5]
eival*eivec:	 [ 1.   0.5 -0.5]


## Gershgorin Circle Theorem

The theorem states that the eigenvalues are going to lie somewhere within the circle around the
diagonal entries where the radius is the absolute sum of the remaining entries of the corresponding row.

### Proof

***The proof is by magic.***

I'm joking. *But if you close your eyes...*


Let us start from the definition of the eigenvalue $\lambda$ and eigenvector $\vec{v}$:
$A\vec{v} = \lambda\vec{v}$. Let $i$ be the index of the entry in $\vec{v}$ with the largest magnitude.

We can write this expression as a summation where we'll get

$$ \sum_{c=1}^n A_{ic}\vec{v}_c = \lambda\vec{v}_i $$
where $A_{ic}$ is the entry on row $i$ and column $c$, and $\vec{v}_i$ is the $i^{th}$ entry of $\vec{v}$.

We can then further split this up into more parts

$$ A_{ii}\vec{v}_i + \sum_{c\neq i} A_{ic}\vec{v}_c = \lambda\vec{v}_i $$

We can then manipulate this a bit to get

$$ \sum_{c\neq i} A_{ic}\vec{v}_c = \lambda\vec{v}_i - A_{ii}\vec{v}_i $$
$$ \sum_{c\neq i} A_{ic}\vec{v}_c = (\lambda - A_{ii})\vec{v}_i $$
$$ \sum_{c\neq i} A_{ic}\frac{\vec{v}_c}{\vec{v}_i} = \lambda - A_{ii} $$

We can take the absolute of both sides and get an inequality from the property of absolutes.

$$ |\lambda - A_{ii}| = \sum_{c\neq i} |A_{ic}||\frac{\vec{v}_c}{\vec{v}_i}| $$

Since $\vec{v}_i$ is the entry with the largest magnitude of $\vec{v}$,
$$\frac{\vec{v}_c}{\vec{v}_i} \leq 1 \forall c$$

As a result, we are going to ignore the term wholely because this is already an inequality.
Finally, we will get 
$$ |\lambda - A_{ii}| = \sum_{c\neq i} |A_{ic}| $$
which can be translated into the distance between $\lambda$ and the diagonal entry of $A$ is
going to be less than or equal to the sum of the non-diagonal entries of the corresponding row.

### Code

In [821]:
def abssum_except(lst, idx):
	return np.sum([ abs(l) for i, l in enumerate(lst) if i != idx ])

def gersh_circle(A:np.ndarray):
	n, _ = A.shape

	alphas = []
	for i in range(n):
		diag_entry = A[i, i]
		rad = abssum_except(A[i], i)
		alpha = rd.random()*2*rad + diag_entry - rad
		alphas.append(alpha-rd.random())
	
	return alphas

In [822]:
gersh_circle(test_matrix)

[-5.275986447836338, 9.65154693212159, -10.79076338617846]

In [823]:
print(np.linalg.eigvals(test_matrix))
for a in gersh_circle(test_matrix):
	print(shifted_eig(test_matrix, a))

[ 1.+0.00000000e+00j -3.+2.71947991e-15j -3.-2.71947991e-15j]
(1.0, array([ 1. ,  0.5, -0.5]))
(-3.0, array([ 1.        ,  0.42857143, -0.71428571]))
(-3.0, array([ 1.        ,  0.42857143, -0.71428571]))


## Putting everything together

Now that we've got (hopefully) everything that we need, we can finally bake our own home-cooked
eigenvalue and eigenvector finder!

In [824]:
def eigen_seeker(A:np.ndarray) -> Tuple[List[float], List[np.ndarray]]:
	eigvals = []
	eigvecs = []
	alphas = gersh_circle(A)

	for a in alphas:
		ei_val, ei_vec = shifted_eig(A, a)
		if ei_val not in eigvals:
			eigvals.append(ei_val)
			eigvecs.append(ei_vec)

	return eigvals, eigvecs

In [825]:
eigen_seeker(test_matrix)

([1.0, -3.0],
 [array([ 1. ,  0.5, -0.5]), array([ 1.        ,  0.42857143, -0.71428571])])

In [826]:
eigvals, eigvecs = eigen_seeker(test_matrix)

print("Numpy eigenvalue:\t", np.linalg.eigvals(test_matrix))
print("Home-baked eigvals:\t", eigvals)
for va, ve in zip(eigvals, eigvecs):
	print()
	print("Eigenvector:\t", ve)
	print("Eigenvalue: \t", va)
	print("A * Eivec:\t", np.matmul(test_matrix, ve))
	print("Eival * Eivec:\t", va*ve)

Numpy eigenvalue:	 [ 1.+0.00000000e+00j -3.+2.71947991e-15j -3.-2.71947991e-15j]
Home-baked eigvals:	 [1.0, -3.0]

Eigenvector:	 [ 1.   0.5 -0.5]
Eigenvalue: 	 1.0
A * Eivec:	 [ 1.   0.5 -0.5]
Eival * Eivec:	 [ 1.   0.5 -0.5]

Eigenvector:	 [ 1.          0.42857143 -0.71428571]
Eigenvalue: 	 -3.0
A * Eivec:	 [-3.         -1.28571429  2.14285714]
Eival * Eivec:	 [-3.         -1.28571429  2.14285714]


In [827]:
A = np.array([
	[2., 0, 0],
	[0., -1, 2],
	[5., 6, -20]
])
eigvals, eigvecs = eigen_seeker(A)

print("Numpy eigenvalue:\t", np.linalg.eigvals(A))
print("Home-baked eigvals:\t", eigvals)
for va, ve in zip(eigvals, eigvecs):
	print()
	print("Eigenvector:\t", ve)
	print("Eigenvalue: \t", va)
	print("A * Eivec:\t", np.matmul(A, ve))
	print("Eival * Eivec:\t", va*ve)

Numpy eigenvalue:	 [-20.61187421  -0.38812579   2.        ]
Home-baked eigvals:	 [2.0, -0.388126, -20.611874]

Eigenvector:	 [1.         0.09842976 0.25123607]
Eigenvalue: 	 2.0
A * Eivec:	 [2.         0.40404239 0.56585708]
Eival * Eivec:	 [2.         0.19685951 0.50247215]

Eigenvector:	 [3.24053459e-17 1.00000000e+00 3.05937104e-01]
Eigenvalue: 	 -0.388126
A * Eivec:	 [ 6.48106917e-17 -3.88125792e-01 -1.18742081e-01]
Eival * Eivec:	 [-1.25773573e-17 -3.88126000e-01 -1.18742144e-01]

Eigenvector:	 [ 1.70586309e-20 -1.01979035e-01  1.00000000e+00]
Eigenvalue: 	 -20.611874
A * Eivec:	 [ 3.41172618e-20  2.10197903e+00 -2.06118742e+01]
Eival * Eivec:	 [-3.51610351e-19  2.10197901e+00 -2.06118740e+01]


## Conclusion

While these may still not be so good/accurate, I think it is good enough for a home-baked one.
We'll end up using the professional one by Numpy or some other library in the future anyway.

Below is some space you can try it out. Have fun and thank you for checking this project out!